# Connect to Indy by IndyDCP

* **robot_ip**: IP address of robot (actually, STEP in IndyCB)
* **name**: string represents robot model
    * Indy7: "NRMK-Indy7"
    * IndyRP2: "NRMK-IndyRP2"
    


In [1]:
from indy_utils import indydcp_client as client
from indy_utils.indy_program_maker import JsonProgramComponent

import json
import threading
from time import sleep

# Set robot (server) IP 
robot_ip = "141.223.199.180"  # Robot (Indy) IP

# Set robot name
name = "NRMK-Indy7"  # Robot name (Indy7)
# name = "NRMK-IndyRP2"  # Robot name (IndyRP2)

# Create class object
indy = client.IndyDCPClient(robot_ip, name)

In [2]:
# Connect to robot
indy.connect()

# Disconnect with robot
# recommanded for finishing program properly
indy.disconnect()

Connect: Server IP (141.223.199.180)


# Examples of JSON Program maker using extended IndyDCP

## Make program example-0: initial and terminal conditions of a program
### Initial statement
* **prog = JsonProgramComponent(policy=0, resume_time=2)**
    * policy: collision policy (0: keep pause, 1: resume after 'resume_time', 2: stop program, 3: no collision detection)
    
### Program body
* **prog.add_move_home()** : go to home position
* **prog.add_move_zero()** : go to zero position

### Terminal statement
* **prog_json = prog.program_done()**  : finish to make program, and return JSON string to be sent to 'indy'
* **indy.set_and_start_json_program(prog_json)** : start the made program
* **indy.set_and_start_json_program(prog.program_done())**  : more simple way

In [3]:
indy.connect()

indy.set_joint_vel_level(3)

prog = JsonProgramComponent(policy=1, resume_time=2)  # Init. prgoram

prog.add_move_home()
prog.add_move_zero()
prog.add_move_home()

prog_json = prog.program_done()    # Program end

indy.set_and_start_json_program(prog_json) # Execute program

indy.disconnect()

Connect: Server IP (141.223.199.180)


In [4]:
indy.connect()

indy.set_joint_vel_level(3)

prog = JsonProgramComponent(policy=1, resume_time=2)  # Init. prgoram

prog.add_move_home()
prog.add_endtool_do(type=0, value=0)
prog.add_move_zero()
prog.add_wait(1)
prog.add_move_home()
prog.add_endtool_do(type=0, value=1)

prog_json = prog.program_done()

indy.set_and_start_json_program(prog_json) # Execute program

indy.disconnect()

Connect: Server IP (141.223.199.180)


## Make program example-0: get teaching positions (joint/task) and save it
* __indy.get_joint_pos()__  : get current joint angle (degree)
* __indy.get_task_pos()__  : get current task-space pose (meter, and degree)

In [5]:
indy.connect()

Connect: Server IP (192.168.3.134)


True

In [6]:
print(indy.get_joint_pos())
print(indy.get_task_pos())

[89.9968235152047, 4.628970376704431e-16, -89.99882348905999, -3.286173052283782e-15, -90.00182344984296, 4.116204156532818e-16]
[0.18651940361205846, 0.3499870854095014, 0.5220071869108897, 1.7312906772995156e-14, -179.99935306109705, 89.9968235152047]


* **indy.update_teaching_data(file_name, waypoint_name, position)**
    * Add teaching points and save it as a file
    * You MUST properly choose joint_move or task_move.
    * We recommand prefix 'j_' or 't_' to represent joint and task

In [7]:
file_name = 'test.json'
teach_config = indy.update_teaching_data(file_name, 'j_wp1', indy.get_joint_pos())
teach_config = indy.update_teaching_data(file_name, 't_wp1', indy.get_task_pos())

In [10]:
teach_config = indy.update_teaching_data(file_name, 'j_wp2', indy.get_joint_pos())
teach_config = indy.update_teaching_data(file_name, 't_wp2', indy.get_task_pos())

In [11]:
teach_config = indy.update_teaching_data(file_name, 'j_wp3', indy.get_joint_pos())
teach_config = indy.update_teaching_data(file_name, 't_wp3', indy.get_task_pos())

In [8]:
teach_config = indy.update_teaching_data(file_name, 'j_wp4', indy.get_joint_pos())
teach_config = indy.update_teaching_data(file_name, 't_wp4', indy.get_task_pos())

In [7]:
indy.disconnect()

## Make program example-1: Joint move with several waypoints
### Arguments: 
* Robot velocity level: 1 - 9
* Joint move blending radius: 3-27 [deg]

In [2]:
file_name = 'test.json'
teach_config = indy.load_teaching_data(file_name)

teach_config

{'t_prepick': [0.35009765625,
  -0.1865234375,
  0.52197265625,
  180.0,
  6.556510925292969e-07,
  90.0],
 't_wp1': [0.4677734375,
  -0.14111328125,
  0.3837890625,
  174.0,
  2.912109375,
  143.125],
 'j_wp1': [4.3125, -13.5390625, -96.25, 2.88671875, -64.0625, 39.875],
 'j_wp2': [-20.53125, -12.9921875, -88.75, 5.125, -73.8125, 14.890625],
 'j_wp3': [6.69140625, -17.859375, -96.75, 2.712890625, -59.15625, 42.03125],
 'j_wp4': [37.90625, -8.046875, -95.125, -1.3134765625, -70.125, 74.875],
 't_wp2': [0.370849609375,
  -0.31640625,
  0.4248046875,
  174.0,
  2.91796875,
  143.125],
 't_wp3': [0.494873046875,
  -0.12017822265625,
  0.348388671875,
  174.0,
  2.94921875,
  143.25],
 't_wp4': [0.457763671875,
  0.11383056640625,
  0.443359375,
  173.875,
  2.96875,
  143.25]}

In [3]:
j_wp1 = teach_config['j_wp1']
j_wp2 = teach_config['j_wp2']
j_wp3 = teach_config['j_wp3']
j_wp4 = teach_config['j_wp4']

t_wp1 = teach_config['t_wp1']
t_wp2 = teach_config['t_wp2']
t_wp3 = teach_config['t_wp3']
t_wp4 = teach_config['t_wp4']

In [4]:
# Joint Move
indy.connect()

vel = 5
blend = 20

prog = JsonProgramComponent(policy=0, resume_time=2)                
prog.add_joint_move_to(j_wp1, vel=vel, blend=blend)
prog.add_joint_move_to(j_wp2, vel=vel, blend=blend)
prog.add_joint_move_to(j_wp3, vel=vel, blend=blend)
prog.add_joint_move_to(j_wp4, vel=vel, blend=blend)

prog_json = prog.program_done()
indy.set_and_start_json_program(prog_json)

# indy.disconnect()

Connect: Server IP (192.168.3.134)


In [ ]:
# indy.stop_current_program()
# indy.pause_current_program()
# indy.resume_current_program()

In [5]:
indy.disconnect()

## Make program example-2: Task move with several waypoints
### Arguments: 
* Robot velocity level: 1 - 9
* Task move blending radius: 0.02 - 0.2 [mm]

In [6]:
# Task Move
indy.connect()

vel = 5
blend = 0.2

prog = JsonProgramComponent(policy=0, resume_time=2)                
prog.add_task_move_to(t_wp1, vel=vel, blend=blend)
prog.add_task_move_to(t_wp2, vel=vel, blend=blend)
prog.add_task_move_to(t_wp3, vel=vel, blend=blend)
prog.add_task_move_to(t_wp4, vel=vel, blend=blend)

indy.set_and_start_json_program(prog.program_done())

indy.disconnect()

Connect: Server IP (192.168.3.134)


## Make program example-3: Use of digital output and tool command


In [7]:
# Simple example
indy.connect()

vel = 5
j_blend = 20
t_blend = 0.2

prog = JsonProgramComponent(policy=0, resume_time=2)

prog.add_move_home()  
prog.add_joint_move_to(j_wp1, vel=vel) 

# Turns on digital output of port indices from 0 to 7 (0: OFF, 1: ON)
for idx in range(0, 8):
    prog.add_digital_out(idx=idx, val=1)

# Wait for set time
prog.add_wait(1)

# Tool command of tool ID and its command
# Tool should be first set up in Conty Application (Setting - Tool)
# In Conty,add tool and set application (e.g. Pick & Place)
# Edit Hold and Release Output and update the tool (refer the Indy manual)
prog.add_endtool_do(type=0, value=0)
prog.add_wait(1)
prog.add_endtool_do(type=0, value=1)

prog.add_task_move_to(t_wp2, vel=vel, blend=t_blend)
prog.add_task_move_to(t_wp3, vel=vel, blend=t_blend)
prog.add_task_move_to(t_wp4, vel=vel, blend=t_blend)

# Turns off digital output of port indices from 0 to 7 (0: OFF, 1: ON)
for idx in range(0, 8):
    prog.add_digital_out(idx=idx, val=0)

prog.add_stop()  # Stop program

indy.set_and_start_json_program(prog.program_done())

indy.disconnect()

Connect: Server IP (192.168.3.134)


## Make program example-4: Sync motion and async motion
* **indy.wait_for_program_finish() :** this command can synchronize several robot programs

In [8]:
# Async and sync example
indy.connect()

vel = 5
blend = 20

# Program loop 2 times
for i in range(0,2):
    # Syncronization with Second program after initail loop
    # Wait until Second program is finished with above command (getting postion)
    indy.wait_for_program_finish()

    # First program
    prog = JsonProgramComponent(policy=0, resume_time=2)
    prog.add_joint_move_to(j_wp2, vel=vel, blend=blend)
    prog.add_move_home()
    
    indy.set_and_start_json_program(prog.program_done())    
    
    # Asyncronization with Second program
    # Wait until First program is finished, after then below program is executed
    indy.wait_for_program_finish()

    # Second program
    prog = JsonProgramComponent(policy=0, resume_time=2)                
    prog.add_task_move_to(t_wp3, vel=vel, blend=blend)
    prog.add_move_home()

    indy.set_and_start_json_program(prog.program_done())
    
indy.disconnect()

Connect: Server IP (192.168.3.134)
Program Finished:  True
Program Finished:  True
Program Finished:  True
Program Finished:  True


## Make program example-5: Multi-threading for interfacing other devices (buttons, sensors,...)

In [2]:
indy.connect()

Connect: Server IP (141.223.199.180)


True

In [3]:
# Global variables
GLOBAL_INDICATOR = {'position': True, 'dio': True, 'program': True, 'terminated': False}


# Monitoring current robot position at every 1 sec 
def monitor_robot_position():
    global GLOBAL_INDICATOR
    while GLOBAL_INDICATOR['position']:
        if GLOBAL_INDICATOR['terminated']:
            break
        sleep(0.1)
        j_pos = indy.get_joint_pos()
        t_pos = indy.get_task_pos()
        print("Joint angles: ", j_pos)
        print("Task pose", t_pos)        
        
# Monitoring digital input at every 1 sec 
def monitor_dio():
    global GLOBAL_INDICATOR
    while GLOBAL_INDICATOR['dio']:
        if GLOBAL_INDICATOR['terminated']:
            break
        sleep(0.1)
        dis = indy.get_di()
        
        btn1 = dis[1]
        btn2 = dis[3]
        btn3 = dis[5]
        print("Button 1/2/3 = [{}, {}, {}]".format(btn1, btn2, btn3))
        if btn1 == 1:
            # TODO: implement an action when button1 on
            pass
        if btn2 == 1:
            # TODO: implement an action when button2 on
            pass
        if btn3 == 1:
            # TODO: implement an action when button3 on
            pass

# Inifinity loop of robot program 
def run_program():
    global GLOBAL_INDICATOR
    while GLOBAL_INDICATOR['program']:        
        if GLOBAL_INDICATOR['terminated']:
            break
        
        prog = JsonProgramComponent(policy=0, resume_time=2)
        
        prog.add_move_home()
        prog.add_move_zero()
        
        indy.set_and_start_json_program(prog.program_done())
        indy.wait_for_program_finish()
        
    GLOBAL_INDICATOR['position'] = False
    GLOBAL_INDICATOR['dio'] = False
    GLOBAL_INDICATOR['program'] = False
    
        
# Create each thread for 'run_program' and 'get_current_position'
th1 = threading.Thread(target=run_program)  # thread for 'run_program'
th2 = threading.Thread(target=monitor_robot_position)  # thread for 'get_current_position'
th3 = threading.Thread(target=monitor_dio)  # thread for 'get_current_position'

th3.start()
th2.start()
th1.start()

Button 1/2/3 = [0, 0, 0]
Joint angles:  [-1.216559328548656e-17, -0.1709672480904656, -1.0258034885427953, -4.967269119970201e-16, -0.8548362404523283, 5.031326368611386e-16]
Task pose [0.04196586527977599, -0.18649999999999997, 1.3266002881705172, 180.0, 174.87615401771478, 179.99999999999997]
Button 1/2/3 = [0, 0, 0]
Button 1/2/3 = [0, 0, 0]
Joint angles:  [-8.590126983569063e-17, -1.5473801269172411, -9.284280761503462, 1.7448591836590315e-16, -7.73690063458622, -9.808056997563282e-17]
Task pose [0.24944024935072742, -0.18649999999999997, 1.2763587719361, 180.0, 148.5849415203238, 179.99999999999997]
Button 1/2/3 = [0, 0, 0]
Button 1/2/3 = [0, 0, 0]
Joint angles:  [-2.9390196636504614e-16, -4.569587535098284, -27.417525210589666, 2.8662372228181653e-15, -22.847937675491373, -2.5149596637737463e-15]
Task pose [0.47851936081000684, -0.1865, 1.1035227959773564, 179.99999999999997, 112.14800581076766, 179.99999999999994]
Button 1/2/3 = [0, 0, 0]
Button 1/2/3 = [0, 0, 0]
Joint angles:  [

In [4]:
# Stop above program
indy.stop_motion()

GLOBAL_INDICATOR['position'] = False
GLOBAL_INDICATOR['dio'] = False
GLOBAL_INDICATOR['program'] = False
GLOBAL_INDICATOR['terminated'] = True

Button 1/2/3 = [0, 0, 0]
Joint angles:  [-1.6201848505412956e-16, -11.815494747828984, -70.89296848697391, -7.385893702997178e-16, -59.07747373914508, -2.7773506041153577e-16]
Task pose [0.602508222362665, -0.18650000000000003, 0.7126284648954763, 2.5195117456403774e-14, 125.29445676721382, 1.841242983565989e-14]
Button 1/2/3 = [0, 0, 0]
Joint angles:  [-2.409863055945353e-16, -8.778999506323151, -52.673997037938946, 6.54910419339819e-16, -43.894997531615765, -1.1605116514789522e-15]
Task pose [0.5763353059964695, -0.1865, 0.9272797701428545, 2.0648740470347894e-13, 94.60221265653745, 1.99641926470458e-13]
Button 1/2/3 = [0, 0, 0]


In [5]:
indy.disconnect()

## Other functions: Direct variables

In [13]:
# Simple variable assignment example
indy.connect()

# Variables
var_name=["ab","bc","ca"]
var_value=[1,2,3]
var_type=[1,1,1]

prog = JsonProgramComponent(policy=0, resume_time=2, var_name=var_name, var_value=var_value, var_type=var_type)

prog.add_move_zero()
prog.add_var_assign(name="ab", val=100, type=1)  # Variable declaration
prog.add_analog_out(idx=0, val=0)

prog.add_wait_for(time=3, left_type=10, left_value="ab", right_type=1, right_value=100, op=0)  # Wait for variable assigned to other value
prog.add_move_home()

indy.set_and_start_json_program(prog.program_done())

indy.disconnect()

Connect: Server IP (192.168.3.134)


# Practice now ! 